# ⚽ Kalman Filter Ball Chase

The red ball uses a Kalman filter to predict where the blue ball will be. The blue ball moves in a circular path while the Kalman filter learns its trajectory and guides the red ball to intercept it.

Adjust the measurement noise to see how the filter adapts to sensor uncertainty!

## Interactive HTML5 Canvas Animation

Run this cell to see the animation with full JavaScript controls!

In [ ]:
from IPython.display import HTML

html_code = '''
<!DOCTYPE html>
<html>
<head>
    <style>
        body {
            background-color: #1a1a2e;
            margin: 0;
            padding: 20px;
            font-family: Arial, sans-serif;
        }
        #canvas-container {
            text-align: center;
        }
        canvas {
            background-color: #1a1a2e;
            border: 2px solid #667eea;
            border-radius: 10px;
            cursor: pointer;
        }
        .controls {
            margin: 20px auto;
            max-width: 700px;
            background: #2a2a3e;
            padding: 20px;
            border-radius: 10px;
        }
        button {
            background: #667eea;
            color: white;
            border: none;
            padding: 10px 20px;
            margin: 5px;
            border-radius: 5px;
            cursor: pointer;
            font-size: 14px;
        }
        button:hover { background: #5568d3; }
        .slider-container {
            margin: 15px 0;
            color: white;
        }
        input[type="range"] {
            width: 100%;
            margin: 10px 0;
        }
        label {
            display: block;
            margin-bottom: 5px;
            font-weight: bold;
        }
    </style>
</head>
<body>
    <div id="canvas-container">
        <canvas id="canvas" width="700" height="700"></canvas>
    </div>
    <div class="controls">
        <div>
            <button onclick="resetSimulation()">🔄 Reset</button>
            <button onclick="togglePause()">⏸️ Pause</button>
        </div>
        <div class="slider-container">
            <label>Measurement Noise (σ²): <span id="noise-value">40</span></label>
            <input type="range" id="noise-slider" min="1" max="150" value="40" 
                   oninput="updateNoise(this.value)">
        </div>
        <div class="slider-container">
            <label>Circle Radius: <span id="radius-value">120</span> px</label>
            <input type="range" id="radius-slider" min="50" max="200" step="10" value="120" 
                   oninput="updateRadius(this.value)">
        </div>
    </div>

    <script>
        const canvas = document.getElementById('canvas');
        const ctx = canvas.getContext('2d');
        
        let paused = false;
        
        // Kalman Filter Class
        class KalmanFilter2D {
            constructor(processNoise = 0.1, measurementNoise = 40) {
                this.state = [350, 350, 0, 0]; // [x, y, vx, vy]
                this.P = [
                    [1000, 0, 0, 0],
                    [0, 1000, 0, 0],
                    [0, 0, 100, 0],
                    [0, 0, 0, 100]
                ];
                this.Q = [
                    [processNoise, 0, 0, 0],
                    [0, processNoise, 0, 0],
                    [0, 0, processNoise * 10, 0],
                    [0, 0, 0, processNoise * 10]
                ];
                this.R = [
                    [measurementNoise, 0],
                    [0, measurementNoise]
                ];
                this.dt = 0.016;
                this.F = [
                    [1, 0, this.dt, 0],
                    [0, 1, 0, this.dt],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]
                ];
                this.H = [
                    [1, 0, 0, 0],
                    [0, 1, 0, 0]
                ];
            }
            
            predict() {
                this.state = this.matMulVec(this.F, this.state);
                this.P = this.matAdd(this.matMul(this.matMul(this.F, this.P), this.transpose(this.F)), this.Q);
            }
            
            update(measurement) {
                const innovation = [measurement[0] - this.state[0], measurement[1] - this.state[1]];
                const S = this.matAdd(this.matMul(this.matMul(this.H, this.P), this.transpose(this.H)), this.R);
                const K = this.matMul(this.matMul(this.P, this.transpose(this.H)), this.inverse2x2(S));
                const correction = this.matMulVec(K, innovation);
                this.state = this.state.map((s, i) => s + correction[i]);
                const I = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]];
                this.P = this.matMul(this.matSub(I, this.matMul(K, this.H)), this.P);
            }
            
            setMeasurementNoise(noise) {
                this.R = [[noise, 0], [0, noise]];
            }
            
            // Matrix operations
            matMul(A, B) {
                const result = [];
                for(let i = 0; i < A.length; i++) {
                    result[i] = [];
                    for(let j = 0; j < B[0].length; j++) {
                        result[i][j] = 0;
                        for(let k = 0; k < B.length; k++)
                            result[i][j] += A[i][k] * B[k][j];
                    }
                }
                return result;
            }
            
            matMulVec(A, v) {
                return A.map(row => row.reduce((sum, val, i) => sum + val * v[i], 0));
            }
            
            matAdd(A, B) {
                return A.map((row, i) => row.map((val, j) => val + B[i][j]));
            }
            
            matSub(A, B) {
                return A.map((row, i) => row.map((val, j) => val - B[i][j]));
            }
            
            transpose(A) {
                return A[0].map((_, i) => A.map(row => row[i]));
            }
            
            inverse2x2(m) {
                const det = m[0][0] * m[1][1] - m[0][1] * m[1][0];
                return [[m[1][1]/det, -m[0][1]/det], [-m[1][0]/det, m[0][0]/det]];
            }
        }
        
        // Simulation State
        let sim = {
            center: {x: 350, y: 350},
            radius: 120,
            blueAngle: 0,
            blueSpeed: 0.008,
            blueRadius: 12,
            redPos: {x: 350 - 120, y: 350},
            redRadius: 12,
            blueTrail: [],
            redTrail: [],
            predTrail: [],
            maxTrail: 200,
            caught: false,
            catchDistance: 25,
            measurementNoise: 40,
            kalman: null
        };
        
        sim.redSpeed = sim.radius * sim.blueSpeed;
        sim.kalman = new KalmanFilter2D(0.1, sim.measurementNoise);
        
        function resetSimulation() {
            sim.blueAngle = 0;
            sim.redPos = {x: sim.center.x - sim.radius, y: sim.center.y};
            sim.blueTrail = [];
            sim.redTrail = [];
            sim.predTrail = [];
            sim.caught = false;
            sim.redSpeed = sim.radius * sim.blueSpeed;
            sim.kalman = new KalmanFilter2D(0.1, sim.measurementNoise);
        }
        
        function togglePause() {
            paused = !paused;
            document.querySelector('button[onclick="togglePause()"]').textContent = 
                paused ? '▶️ Resume' : '⏸️ Pause';
        }
        
        function updateNoise(value) {
            sim.measurementNoise = parseFloat(value);
            sim.kalman.setMeasurementNoise(sim.measurementNoise);
            document.getElementById('noise-value').textContent = value;
        }
        
        function updateRadius(value) {
            sim.radius = parseFloat(value);
            document.getElementById('radius-value').textContent = value;
            resetSimulation();
        }
        
        function update() {
            if (paused) return;
            
            // Update blue ball
            sim.blueAngle += sim.blueSpeed;
            const bluePos = {
                x: sim.center.x + sim.radius * Math.cos(sim.blueAngle),
                y: sim.center.y + sim.radius * Math.sin(sim.blueAngle)
            };
            
            // Add measurement noise
            const noise = {
                x: (Math.random() - 0.5) * 2 * Math.sqrt(sim.measurementNoise),
                y: (Math.random() - 0.5) * 2 * Math.sqrt(sim.measurementNoise)
            };
            const measuredPos = [bluePos.x + noise.x, bluePos.y + noise.y];
            
            // Kalman filter
            sim.kalman.predict();
            sim.kalman.update(measuredPos);
            
            // Predict future position
            const futureSteps = 5;
            const futureAngle = sim.blueAngle + sim.blueSpeed * futureSteps;
            const predPos = {
                x: sim.center.x + sim.radius * Math.cos(futureAngle),
                y: sim.center.y + sim.radius * Math.sin(futureAngle)
            };
            
            // Red ball moves toward prediction
            const dx = predPos.x - sim.redPos.x;
            const dy = predPos.y - sim.redPos.y;
            const dist = Math.sqrt(dx*dx + dy*dy);
            
            if (dist > 0) {
                sim.redPos.x += (dx/dist) * sim.redSpeed;
                sim.redPos.y += (dy/dist) * sim.redSpeed;
            }
            
            // Check caught
            const catchDist = Math.sqrt(
                Math.pow(bluePos.x - sim.redPos.x, 2) + 
                Math.pow(bluePos.y - sim.redPos.y, 2)
            );
            sim.caught = catchDist < sim.catchDistance;
            
            // Update trails
            sim.blueTrail.push({...bluePos});
            sim.redTrail.push({...sim.redPos});
            sim.predTrail.push({...predPos});
            
            if (sim.blueTrail.length > sim.maxTrail) {
                sim.blueTrail.shift();
                sim.redTrail.shift();
                sim.predTrail.shift();
            }
            
            return {bluePos, predPos, catchDist};
        }
        
        function draw() {
            const {bluePos, predPos, catchDist} = update() || {};
            
            // Clear canvas
            ctx.fillStyle = '#1a1a2e';
            ctx.fillRect(0, 0, canvas.width, canvas.height);
            
            // Draw circular path
            ctx.strokeStyle = 'rgba(255, 255, 255, 0.3)';
            ctx.lineWidth = 2;
            ctx.setLineDash([5, 5]);
            ctx.beginPath();
            ctx.arc(sim.center.x, sim.center.y, sim.radius, 0, Math.PI * 2);
            ctx.stroke();
            ctx.setLineDash([]);
            
            // Draw trails
            if (sim.blueTrail.length > 1) {
                ctx.strokeStyle = 'rgba(52, 152, 219, 0.3)';
                ctx.lineWidth = 1.5;
                ctx.beginPath();
                ctx.moveTo(sim.blueTrail[0].x, sim.blueTrail[0].y);
                for(let i = 1; i < sim.blueTrail.length; i++) {
                    ctx.lineTo(sim.blueTrail[i].x, sim.blueTrail[i].y);
                }
                ctx.stroke();
                
                ctx.strokeStyle = 'rgba(231, 76, 60, 0.3)';
                ctx.beginPath();
                ctx.moveTo(sim.redTrail[0].x, sim.redTrail[0].y);
                for(let i = 1; i < sim.redTrail.length; i++) {
                    ctx.lineTo(sim.redTrail[i].x, sim.redTrail[i].y);
                }
                ctx.stroke();
            }
            
            // Draw prediction line
            if (predPos) {
                ctx.strokeStyle = 'rgba(46, 204, 113, 0.5)';
                ctx.lineWidth = 1.5;
                ctx.setLineDash([5, 3]);
                ctx.beginPath();
                ctx.moveTo(sim.redPos.x, sim.redPos.y);
                ctx.lineTo(predPos.x, predPos.y);
                ctx.stroke();
                ctx.setLineDash([]);
                
                // Draw prediction dot
                ctx.fillStyle = 'rgba(46, 204, 113, 0.7)';
                ctx.beginPath();
                ctx.arc(predPos.x, predPos.y, 6, 0, Math.PI * 2);
                ctx.fill();
            }
            
            // Draw balls
            if (bluePos) {
                ctx.fillStyle = '#3498db';
                ctx.beginPath();
                ctx.arc(bluePos.x, bluePos.y, sim.blueRadius, 0, Math.PI * 2);
                ctx.fill();
            }
            
            ctx.fillStyle = '#e74c3c';
            ctx.beginPath();
            ctx.arc(sim.redPos.x, sim.redPos.y, sim.redRadius, 0, Math.PI * 2);
            ctx.fill();
            
            // Draw status
            const statusText = sim.caught ? '🎉 Caught! ✨' : '🎯 Chasing...';
            const statusColor = sim.caught ? '#ff9800' : '#667eea';
            
            ctx.fillStyle = statusColor;
            ctx.fillRect(260, 630, 180, 40);
            ctx.fillStyle = 'white';
            ctx.font = 'bold 16px Arial';
            ctx.textAlign = 'center';
            ctx.fillText(statusText, 350, 655);
            
            // Draw info
            if (catchDist !== undefined) {
                ctx.fillStyle = 'white';
                ctx.font = '10px monospace';
                ctx.fillText(`Distance: ${catchDist.toFixed(1)} px`, 350, 30);
            }
            
            requestAnimationFrame(draw);
        }
        
        // Start animation
        draw();
    </script>
</body>
</html>
'''

HTML(html_code)

## 📐 Kalman Filter: Mathematical Foundation

### The Core Idea: Prediction + Correction

The Kalman filter works in two alternating phases:

- **PREDICT:** "Based on physics and past behavior, where should the ball be now?"
- **UPDATE:** "I measured it at position Z. My prediction was X. Let me blend these intelligently."

### The Algorithm

**PREDICTION PHASE:**

State Prediction:
$$\hat{\mathbf{x}}^-(k) = \mathbf{F} \cdot \hat{\mathbf{x}}(k-1)$$

Covariance Prediction:
$$\mathbf{P}^-(k) = \mathbf{F} \cdot \mathbf{P}(k-1) \cdot \mathbf{F}^T + \mathbf{Q}$$

**UPDATE PHASE:**

Innovation:
$$\mathbf{y}(k) = \mathbf{z}(k) - \mathbf{H} \cdot \hat{\mathbf{x}}^-(k)$$

Innovation Covariance:
$$\mathbf{S}(k) = \mathbf{H} \cdot \mathbf{P}^-(k) \cdot \mathbf{H}^T + \mathbf{R}$$

Kalman Gain:
$$\mathbf{K}(k) = \mathbf{P}^-(k) \cdot \mathbf{H}^T \cdot \mathbf{S}^{-1}(k)$$

Updated State:
$$\hat{\mathbf{x}}(k) = \hat{\mathbf{x}}^-(k) + \mathbf{K}(k) \cdot \mathbf{y}(k)$$

Updated Covariance:
$$\mathbf{P}(k) = (\mathbf{I} - \mathbf{K}(k) \cdot \mathbf{H}) \cdot \mathbf{P}^-(k)$$